<html>
<div style="width:100%">
  <div style="width:90%; float:left; background:white; margin:10px">
    <img style ="width:100%"src ="https://raw.githubusercontent.com/publicunpublic/QCMM_Jupyter/main/full.svg">
    <div style="margin-top:2.5%;position:absolute;background-color:#0e4be5;width:6%;height:0.3%"></div>
  </div>
  </div>


<html>
<center>
<br>
<h1 style="color:#E6610F;"> Check Normal Mode of Transition State and verify Nature of TS </h1>
<br><br>
</center>

In [6]:
import qcportal as ptl
import pandas as pd
import numpy as np
from typing import Union
import qcelemental as qcel

## Define Variables

In [9]:
client_address = "152.74.10.245:7778"
ts_opt_spec = 'b3lyp-d3_def2-svp'
ts_collection = 'salen_d-lac_ts_from_neb'
dft_method = 'b3lyp'
dft_basis = 'def2-svp'
entry_name = 'ts1_01'
username = 'svogt'
password = "7kyRT-Mrow3jH0Lg6b9YIhEjAcvU9EpFBb9ouMClU5g"

In [10]:
client = ptl.FractalClient(address=client_address, username = username, password = password, verify=False)
client 

FractalClient(server_name='Catalisis Server', address='https://152.74.10.245:7778/', username='svogt')

## A) Load Modified Psi4 function for NM analysis

In [11]:
def vibanal_wfn(hess: np.ndarray = None, irrep: Union[int, str] = None, molecule=None, energy=None, project_trans: bool = True, project_rot: bool = True, molden=False,name=None, lt=None):
    """Function to perform analysis of a hessian or hessian block, specifically...
    calling for and printing vibrational and thermochemical analysis, setting thermochemical variables,
    and writing the vibrec and normal mode files.
    Parameters
    ----------
    wfn
        The wavefunction which had its Hessian computed.
    hess
        Hessian to analyze, if not the hessian in wfn.
        (3*nat, 3*nat) non-mass-weighted Hessian in atomic units, [Eh/a0/a0].
    irrep
        The irrep for which frequencies are calculated. Thermochemical analysis is skipped if this is given,
        as only one symmetry block of the hessian has been computed.
    molecule : :py:class:`~psi4.core.Molecule` or qcdb.Molecule, optional
        The molecule to pull information from, if not the molecule in wfn. Must at least have similar
        geometry to the molecule in wfn.
    project_trans
        Should translations be projected in the harmonic analysis?
    project_rot
        Should rotations be projected in the harmonic analysis?
    Returns

    -------
    vibinfo : dict
        A dictionary of vibrational information. See :py:func:`~psi4.driver.qcdb.vib.harmonic_analysis`
    """
    from psi4 import geometry, set_memory
    import psi4.core as core
    from psi4.driver import qcdb
    
    if hess is None:
        print("no hessian")
        
    else:
        nmwhess = hess
        
    set_memory('500 MB')
    
    
    print("About to start")
    m=molecule.to_string('xyz')
    mol = geometry(m)
    geom = np.asarray(mol.geometry())
    symbols = [mol.symbol(at) for at in range(mol.natom())]
    vibrec = {'molecule': mol.to_dict(np_out=False), 'hessian': nmwhess.tolist()}

    m = np.asarray([mol.mass(at) for at in range(mol.natom())])
    irrep_labels = mol.irrep_labels()
    
    print(qcdb.vib.__file__)
    print("About to do harmonic analyis...")
    vibinfo, vibtext = qcdb.vib.harmonic_analysis(
        nmwhess, geom, m, None, irrep_labels, dipder=None, project_trans=project_trans, project_rot=project_rot)

    print("Finished harmonic analysis")
   

    if core.has_option_changed('THERMO', 'ROTATIONAL_SYMMETRY_NUMBER'):
        rsn = core.get_option('THERMO', 'ROTATIONAL_SYMMETRY_NUMBER')
    else:
        rsn = mol.rotational_symmetry_number()

    if irrep is None:
        therminfo, thermtext = qcdb.vib.thermo(
            vibinfo,
            T=core.get_option("THERMO", "T"),  # 298.15 [K]
            P=core.get_option("THERMO", "P"),  # 101325. [Pa]
            multiplicity=mol.multiplicity(),
            molecular_mass=np.sum(m),
            sigma=rsn,
            rotor_type=mol.rotor_type(),
            rot_const=np.asarray(mol.rotational_constants()),
            E0=energy) 

        core.set_variable("ZPVE", therminfo['ZPE_corr'].data)  # P::e THERMO
        core.set_variable("THERMAL ENERGY CORRECTION", therminfo['E_corr'].data)  # P::e THERMO
        core.set_variable("ENTHALPY CORRECTION", therminfo['H_corr'].data)  # P::e THERMO
        core.set_variable("GIBBS FREE ENERGY CORRECTION", therminfo['G_corr'].data)  # P::e THERMO

        core.set_variable("ZERO K ENTHALPY", therminfo['ZPE_tot'].data)  # P::e THERMO
        core.set_variable("THERMAL ENERGY", therminfo['E_tot'].data)  # P::e THERMO
        core.set_variable("ENTHALPY", therminfo['H_tot'].data)  # P::e THERMO
        core.set_variable("GIBBS FREE ENERGY", therminfo['G_tot'].data)  # P::e THERMO

    else:
        core.print_out('  Thermochemical analysis skipped for partial frequency calculation.\n')

    
    if molden:
        filename = name+"_"+lt+ ".molden_normal_modes"
        with open(filename, 'w') as handle:
            handle.write(qcdb.vib.print_molden_vibs(vibinfo, symbols, geom, standalone=True))

    return vibinfo, therminfo, symbols, geom

## B) Load  Optimized Transition states from Dataset

In [12]:
ds_opt = client.get_collection("OptimizationDataset", ts_collection)

rr = ds_opt.get_record(entry_name, specification = ts_opt_spec)

if rr.status == ("ERROR" or "INCOMPLETE"):
    print(rr.status ,"Optimization not done yet, please be a little more patient")

else:
    print(rr.status, "Optimization finished!")


RecordStatusEnum.complete Optimization finished!


In [13]:
# Get the optimized TS ID

mol_opt_ts = rr.get_final_molecule()
ts_id = mol_opt_ts.id
mol_opt_ts_init = rr.get_initial_molecule()
ts_id_init = mol_opt_ts_init.id
print("Optimized molecule ID:", ts_id)
print("Initial molecule ID:", ts_id_init)

Optimized molecule ID: 30064
Initial molecule ID: 30054


## C) Get the hessian matrix for the respective transition states from the server

In [14]:
record = client.query_results(driver='hessian', 
                              molecule=ts_id, 
                              method=dft_method, 
                              basis=dft_basis 
                              )

In [15]:
record

[ResultRecord(id='38954', status='COMPLETE')]

## D)  Do harmonic analysis for the hessian matrix 

You can visualize the NM in the .molden_normal_modes file

In [17]:
vib_info = {}
hess_mat = record[0].dict()['return_result']
en = record[0].dict()['properties']['return_energy']

vib_info = vibanal_wfn(hess=hess_mat, molecule=mol_opt_ts, name=entry_name, energy=en, lt=ts_opt_spec, molden=True)

About to start
  Memory set to 476.837 MiB by Python driver.

/home/svogt/.anaconda/envs/qcportal/lib/python3.7/site-packages/psi4/driver/qcdb/vib.py
About to do harmonic analyis...
Finished harmonic analysis
 '82.1494' '104.3671' '115.1663' '119.2267' '139.3989' '152.5566'
 '162.7541' '169.6487' '193.0081' '214.0929' '218.6423' '223.3994'
 '232.1711' '239.1127' '253.4529' '260.5176' '271.9467' '276.6946'
 '304.2322' '309.3069' '317.8659' '352.0669' '355.4503' '362.8523'
 '373.5442' '385.3100' '399.5407' '404.4310' '416.8776' '427.8118'
 '442.7413' '447.5931' '475.7286' '483.7059' '503.0203' '505.2721'
 '513.6687' '535.0686' '545.6932' '565.0643' '565.9659' '574.5115'
 '589.5387' '616.9860' '623.9872']


## E)  Get Intermadiate candidates from TS normal mode

In [20]:
import math

nm_num = 0  # Number of the normal mode (Imaginary mode is 0)
vib_range = 1.8 # The range of the vibration

vib, therm, symbols, geom = vib_info
nat = int(len(vib['q'].data[:, nm_num]) / 3)
nm = vib['q'].data[:, nm_num].reshape(nat, 3)#[at].real



disp_list = np.arange(-vib_range, vib_range, 0.1)

close_reac = vib_range - 0.1
close_int  = -vib_range


# Saving the extremes as molecule objects
vib_animation = ""
for d in disp_list:
    disp = (geom - d*nm) 
    
    ## Creating molecule from displacements
    mol = qcel.models.Molecule(
    symbols=symbols,
    geometry = disp,
    )
    
    ## Saving reactant and product candidates 
    vib_animation += mol.to_string("xyz")
    if math.isclose(close_int,d):
        int_cand  = mol
    if math.isclose(close_reac,d):
        reac_cand = mol

    # Saving the 
    #filename = "animts2_"+entry_name+"_"+ts_opt_spec+ ".dat"
    #with open(filename, 'w') as handle:
    #    handle.write(vib_animation)

## F) Visualize Intermediate initial structures 

In [22]:
reac_cand

NGLWidget()

In [23]:
mol_opt_ts

NGLWidget()

In [21]:
int_cand

NGLWidget()

## G)  Optimize intermediate states 

In [97]:
#ds = ptl.collections.OptimizationDataset("salen_d-lac_rpi", client=client)
#ds.save()

In [21]:
ds_opt_rpi = client.get_collection("OptimizationDataset", "salen_d-lac_rpi")

In [24]:
ds_opt_rpi.add_entry(entry_name+'_r', reac_cand)
ds_opt_rpi.add_entry(entry_name+'_i1', int_cand)

In [53]:
kw = ptl.models.KeywordSet(**{"values": {"dftd": "d3", "convthre" : '3.0e-7', "threall" : '1.0e-13', 'dftgrid' : 2, "scf" : "diis+a"}})
kw_id = client.add_keywords([kw])[0]


add_spec = {'name':"b3lyp-d3_def2-svp",
        'description': 'Geometric + TERACHEM/b3lyp-d3/def2-svp.',
        'optimization_spec': {'program': 'geometric', 'keywords': {"converge" : ["set" , "gau_tight"]}},
        'qc_spec': {'driver': 'gradient',
        'method': "b3lyp",
        'basis': 'def2-svp',
        'keywords': kw_id,
        'program': 'terachem'}}
ds_opt_rpi.add_specification(**add_spec,overwrite=True)
ds_opt_rpi.save()

'123'

## G)  Check Optimization 

In [25]:
c = ds_opt_rpi.compute('b3lyp-d3_def2-svp', tag='tera')
print(c)

2


In [26]:
ds_opt_rpi = client.get_collection("OptimizationDataset", "salen_d-lac_rpi")
ds_opt_rpi.status(collapse=False)

,b3lyp-d3_def2-svp
ts1_04_r,COMPLETE
ts1_04_i1,COMPLETE
ts1_02_r,COMPLETE
ts1_02_i1,COMPLETE
ts1_03_r,COMPLETE
ts1_03_i1,COMPLETE
ts2_02_r,COMPLETE
ts2_02_i1,COMPLETE
ts1_01_r,COMPLETE
ts1_01_i1,COMPLETE


In [29]:
rr = ds_opt_rpi.get_record(name='ts1_01_r', specification = "b3lyp-d3_def2-svp")
mol = rr.get_final_molecule()
#print(mol.pretty_print())
mol

NGLWidget()

## Extras

## Load a hessian from a file


In [87]:
#import numpy as np
#h = np.loadtxt("./hessian_1.txt", delimiter = ' ')
#h.shape

### Restart Optimization jobs

In [ ]:
mol_list = []

spec = 'pbe0-d3_def2-svp'

for i in ds_opt.data.records: 
    #print(i)
    mol_list.append(i)

for i in  mol_list:
    opt_rec = ds_opt.get_record(name=i,specification= spec)
    if (opt_rec.status == 'ERROR'):    
        id_u = opt_rec.id 
        print('yes', i, id_u)
        c = client.modify_tasks("modify", new_tag = 'refinement_tera',base_result = id_u)
        #c = client.modify_tasks("restart",base_result = id_u)
        print(c)